In [ ]:
# %% [markdown]
# # Lab 6: Conditional Edges & Smart Routing
#
# **Goal:** Enhance our graph by adding conditional logic, allowing it to take different paths based on the data in the state. This is a key feature for creating truly dynamic and intelligent agents.
#
# ---

# %% [markdown]
# ## Setup
#
# We'll start with the nodes and state from the previous lab.

# %%
# Install required packages
!pip install langgraph langchain-openai --quiet
print("Libraries installed.")

# Basic imports
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

# Define the state
class ResumeState(TypedDict):
    resume_text: str
    skills: List[str]
    score: int
    decision: str

# Re-define nodes from the previous lab
def parse_resume(state: ResumeState) -> dict:
    print("---NODE: PARSING RESUME---")
    resume = state["resume_text"]
    found_skills = []
    if "python" in resume.lower(): found_skills.append("Python")
    if "java" in resume.lower(): found_skills.append("Java")
    if "javascript" in resume.lower(): found_skills.append("JavaScript")
    if "management" in resume.lower(): found_skills.append("Management")
    if "leadership" in resume.lower(): found_skills.append("Leadership")
    return {"skills": found_skills}

def score_candidate(state: ResumeState) -> dict:
    print("---NODE: SCORING CANDIDATE---")
    score = len(state["skills"]) * 20
    print(f"Candidate score: {score}")
    return {"score": score}

print("State and nodes are ready.")

# %% [markdown]
# ---
# ## Part A: The Routing Function
#
# A conditional edge uses a special function to decide which node to go to next. This function inspects the current `state` and returns a string that matches the name of the desired path.
#
# Our routing function will look at the `score` and decide if the candidate should be "fast_tracked", sent for "standard_review", or "rejected".

# %%
def route_candidate(state: ResumeState) -> str:
    """
    This function decides which path to take based on the candidate's score.
    It returns a string name of the path.
    """
    score = state["score"]
    print(f"---ROUTING: Score is {score}---")

    if score >= 80:
        print("Decision: Fast Track")
        return "fast_track"
    elif score >= 40:
        print("Decision: Standard Review")
        return "standard_review"
    else:
        print("Decision: Reject")
        return "auto_reject"

# %% [markdown]
# ---
# ## Part B: Building the Enhanced Graph
#
# 1.  **Add New Nodes:** We'll create a specific node for each of our new paths (`fast_track_process`, `standard_review`, `auto_reject`).
# 2.  **Add Conditional Edge:** We will use `add_conditional_edges`. This special method connects a source node (`score`) to our routing function (`route_candidate`) and provides a mapping from the function's return values to the destination nodes.

# %%
# 1. Add new nodes for the different paths
def fast_track_process(state: ResumeState) -> dict:
    """Node for high-scoring candidates."""
    print("---NODE: FAST TRACK---")
    return {"decision": "Schedule Technical Interview Immediately"}

def standard_review(state: ResumeState) -> dict:
    """Node for medium-scoring candidates."""
    print("---NODE: STANDARD REVIEW---")
    return {"decision": "HR Manual Review Required"}

def auto_reject(state: ResumeState) -> dict:
    """Node for low-scoring candidates."""
    print("---NODE: AUTO REJECT---")
    return {"decision": "Rejected - Does not meet minimum requirements"}

# 2. Build the new graph with conditional logic
workflow = StateGraph(ResumeState)

# Add all the nodes
workflow.add_node("parse", parse_resume)
workflow.add_node("score", score_candidate)
workflow.add_node("fast_track", fast_track_process)
workflow.add_node("standard_review", standard_review)
workflow.add_node("auto_reject", auto_reject)

# Add the standard edges
workflow.set_entry_point("parse")
workflow.add_edge("parse", "score")

# 3. Add the conditional edge
workflow.add_conditional_edges(
    "score",          # The source node
    route_candidate,  # The function that decides the path
    {
        # The mapping from the function's return string to the destination node
        "fast_track": "fast_track",
        "standard_review": "standard_review",
        "auto_reject": "auto_reject"
    }
)

# All paths eventually lead to the end
workflow.add_edge("fast_track", END)
workflow.add_edge("standard_review", END)
workflow.add_edge("auto_reject", END)

# Compile the graph
app = workflow.compile()
print("\nConditional graph compiled successfully!")

# %% [markdown]
# ---
# ## Part C: Testing the Different Scenarios
#
# Now, let's test our graph with different resumes to ensure the routing logic works correctly.

# %%
# Test Case 1: High-scoring candidate
print("---TESTING HIGH-SCORER---")
high_scorer_resume = "Senior Python developer with extensive Java, JavaScript, project management, and team leadership experience."
high_scorer_input = {"resume_text": high_scorer_resume}
result1 = app.invoke(high_scorer_input)
print(f"Final Result: {result1['decision']}\n")


# Test Case 2: Medium-scoring candidate
print("---TESTING MEDIUM-SCORER---")
medium_scorer_resume = "Python developer with some Java experience."
medium_scorer_input = {"resume_text": medium_scorer_resume}
result2 = app.invoke(medium_scorer_input)
print(f"Final Result: {result2['decision']}\n")


# Test Case 3: Low-scoring candidate
print("---TESTING LOW-SCORER---")
low_scorer_resume = "Recent graduate with basic programming knowledge in C++."
low_scorer_input = {"resume_text": low_scorer_resume}
result3 = app.invoke(low_scorer_input)
print(f"Final Result: {result3['decision']}\n")